https://learnopencv.com/video-stabilization-using-point-feature-matching-in-opencv/

In [1]:
import cv2
import sys
import numpy as np
import matplotlib.pyplot as plt

In [2]:
class KalmanFilter(object):
    def __init__(self, F = None, H = None, Q = None, R = None, P = None, x0 = None):

        if(F is None or H is None):
            raise ValueError("Set proper system dynamics.")

        self.n = F.shape[1]
        self.m = H.shape[1]

        self.F = F
        self.H = H

        self.Q = np.eye(self.n) if Q is None else Q
        self.R = np.eye(self.n) if R is None else R
        self.P = np.eye(self.n) if P is None else P
        self.x = np.zeros((self.n, 1)) if x0 is None else x0

    def predict(self):
        self.x = np.matmul(self.F, self.x)
        self.P = np.matmul(np.matmul(self.F, self.P), self.F.transpose()) + self.Q
        return self.x

    def update(self, z):
        y = z - np.matmul(self.H, self.x)
        S = self.R + np.matmul(self.H, np.matmul(self.P, self.H.transpose()))

        K = np.matmul(np.matmul(self.P, self.H.transpose()), np.linalg.inv(S))
        self.x = self.x + np.matmul(K, y)
        I = np.eye(self.n)
        self.P = np.matmul(np.matmul(I - np.matmul(K, self.H), self.P), 
        	(I - np.matmul(K, self.H)).transpose()) + np.matmul(np.matmul(K, self.R), K.transpose())

In [ ]:
alive = True

win_name = "Motion Correction"
cv2.namedWindow(win_name, cv2.WINDOW_NORMAL)
result = None
source = cv2.VideoCapture(0)


has_frame, prev_frame = source.read()
prev_frame = cv2.flip(prev_frame, 1)
prev_frame = cv2.cvtColor(prev_frame,cv2.COLOR_BGR2GRAY)

prev_pts = cv2.goodFeaturesToTrack(prev_frame,maxCorners=200,qualityLevel=0.01,minDistance=30,blockSize=5)
cv2.imshow(win_name, cv2.hconcat([prev_frame,prev_frame]))

kfilt = KalmanFilter(F=np.eye(3),H=np.eye(3))
x = np.zeros((3,1))

while alive:
    has_frame, cur_frame = source.read()
    if not has_frame:
        break

    cur_frame = cv2.flip(cur_frame, 1)
    cur_frame = cv2.cvtColor(cur_frame,cv2.COLOR_BGR2GRAY)

    curr_pts = cv2.goodFeaturesToTrack(prev_frame,maxCorners=200,qualityLevel=0.01,minDistance=30,blockSize=5)
    curr_pts,status,err = cv2.calcOpticalFlowPyrLK(prev_frame,cur_frame,prev_pts,curr_pts) 

    # Use only matching paits
    idx = np.where(status==1)[0]
    
    # Estimate a rigid body transform
    m,_ = cv2.estimateAffinePartial2D(prev_pts[idx],curr_pts[idx])

    x[0,0] = m[0,2] # dX
    x[1,0] = m[1,2] # dY
    x[2,0] = np.arctan2(m[1,0],m[0,0])  # rotation angle
  
    prev_frame=cur_frame
    prev_pts = curr_pts

    # Use the kalman filter to smooth the estimate of the states (dx,dy,rotation angle )
    kfilt.update(x)
    x=kfilt.predict()
    
    # Convert the states back to a rotation matrix
    m[0,2]=x[0,0]
    m[1,2]=x[1,0]
    m[0,0]=np.cos(x[2,0])
    m[0,1]=-np.sin(x[2,0])
    m[1,0]=np.sin(x[2,0])
    m[1,1]=np.cos(x[2,0])
    
    # Add the key points to the image for display purpose
    for id in idx:
        i=np.uint16(curr_pts[id][0][0])
        j=np.uint16(curr_pts[id][0][1])
        cur_frame=cv2.circle(cur_frame,(i,j),3,255,3)

    # Apply the correction
    cur_frame_corrected = cv2.warpAffine(cur_frame,m,(cur_frame.shape[1],cur_frame.shape[0]))

    # show the uncorrected and corrected side-by-side
    cv2.imshow(win_name, cv2.hconcat([cur_frame,cur_frame_corrected]))
    
    key = cv2.waitKey(1)
    if key == ord("Q") or key == ord("q") or key == 27:
        alive = False

[ WARN:0@3.414] global /private/var/folders/sy/f16zz6x50xz3113nwtb9bvq00000gp/T/abs_f6tvh9615u/croot/opencv-suite_1691620375715/work/modules/videoio/src/cap_gstreamer.cpp (862) isPipelinePlaying OpenCV | GStreamer warning: GStreamer: pipeline have not been created
